<a href="https://colab.research.google.com/github/ntatfff/todo/blob/202411241258/ColabRadiomicsFeatureExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py39" --user
!python3 --version

--2024-12-28 00:11:29--  https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61451533 (59M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  58.60M   310MB/s    in 0.2s    

2024-12-28 00:11:30 (310 MB/s) - ‘mini.sh’ saved [61451533/61451533]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - brotlipy==0.7.0=py39h27cfd23_1003
    - ca-certificates==2020.12.8=h06a4308_0
    - certifi==2020.12.5=py39h06a4308_0
    - cffi==1.14.4=py39h261ae71_0
    - chardet==3.0.4=py39h06a4308_1003
    - conda-package-handling==1.7.2=py39h27cfd23_1
    - co

In [3]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/local/lib/python3.9 1

update-alternatives: using /usr/local/lib/python3.9 to provide /usr/bin/python3 (python3) in auto mode


In [4]:
%pip install SimpleITK==2.4.0
%pip install numpy==1.26.4
%pip install pyradiomics==3.1.0
%pip install pandas==1.2.3

     |████████████████████████████████| 52.4 MB 145 kB/s 
     |████████████████████████████████| 18.2 MB 18.9 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.3
    Uninstalling numpy-1.22.3:
      Successfully uninstalled numpy-1.22.3
     |████████████████████████████████| 177 kB 3.2 MB/s 
     |████████████████████████████████| 4.5 MB 20.1 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=21d239110205af42bd30f00ac0f98de5c391950dc14133cd5db55929443e992b
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [5]:
%%writefile code.py

import radiomics
import numpy as np
import SimpleITK as sitk
import radiomics.featureextractor
import os
import six
from os import path
import pandas as pd

def featureExtractor(fileId):
  imagePath = '/content/drive/MyDrive/LuanVan/dataset/BraTS2021_Training_Data/%s/%s_flair.nii.gz' % (fileId, fileId)
  image = sitk.ReadImage(imagePath)
  maskPath ='/content/drive/MyDrive/LuanVan/dataset/BraTS2021_Training_Data/%s/%s_tumor.nii.gz' % (fileId, fileId)
  mask = sitk.ReadImage(maskPath)

  kernel = 5
  settings = {}
  settings['kernelRadius'] = kernel
  settings['voxelBatch'] = 40000
  extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(**settings)
  extractor.disableAllFeatures()
  extractor.enableFeatureClassByName('firstorder')

  featureMap = extractor.execute(image, mask, voxelBased=True)

  for featureName, featureValue in six.iteritems(featureMap):
    if isinstance(featureValue, sitk.Image):
      fileFolder = '/content/drive/MyDrive/LuanVan/dataset/kernel%s/%s' % (kernel, fileId)
      if path.exists(fileFolder) == False:
        os.mkdir(fileFolder)
      sitk.WriteImage(featureValue, '%s/%s.nrrd' % (fileFolder, featureName))
      print('Computed %s, stored as "%s/%s.nrrd"' % (featureName, fileFolder, featureName))
    else:
      print('%s: %s' % (featureName, featureValue))

monitorFilePath = '/content/drive/MyDrive/LuanVan/dataset/kernel5/1.monitor.csv'
monitor = pd.read_csv(monitorFilePath, index_col='no')
for i, row in monitor.iterrows():
    if row['done'] != 0:
        continue
    fileId = row['file']
    print('Starting %s' % (fileId))
    featureExtractor(fileId)
    monitor.at[i, 'done'] = 1
    monitor.to_csv(monitorFilePath)

Writing code.py


In [6]:
!python3 code.py

Starting BraTS2021_00048
/usr/local/lib/python3.9/site-packages/radiomics/firstorder.py:297: RuntimeWarning: Mean of empty slice
  return numpy.nanmean(numpy.absolute(percentileArray - numpy.nanmean(percentileArray, 1, keepdims=True)), 1)
diagnostics_Versions_PyRadiomics: v3.1.0
diagnostics_Versions_Numpy: 1.26.4
diagnostics_Versions_SimpleITK: 2.4.0
diagnostics_Versions_PyWavelet: 1.6.0
diagnostics_Versions_Python: 3.9.21
diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'kernelRadius': 5, 'voxelBatch': 40000, 'voxelBased': True}
diagnostics_Configuration_EnabledImageTypes: {'Original': {}}
diagnostics_Image-original_Hash: fdb9d0474bbf47b09cfb9471014db2fdcd19b5a8
diagnos